# Update net

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from main import CIFAR10_dataset, CIFAR10_trainer
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as  F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

In [3]:
import torchvision.transforms as transforms

def add_gaussian_noise(img, mean=0, std=0.1):
    noise = torch.randn(img.size()) * std + mean
    return img + noise

# Transformaciones para entrenamiento
train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomRotation(degrees=25),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),
    
    transforms.ToTensor(),
    # transforms.RandomErasing(p=0.6, scale=(0.1, 0.5), ratio=(0.4, 4.4)),
    transforms.Lambda(lambda x: add_gaussian_noise(x, 0, 0.2)),
])


train_dataset = CIFAR10_dataset(partition="train", transform=train_transforms)
test_dataset = CIFAR10_dataset(partition="test")

####################################################################
# DataLoader Class
####################################################################

batch_size = 150
num_workers = multiprocessing.cpu_count()-1
print("Num workers", num_workers)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=num_workers)


Loading CIFAR10  train  Dataset...
Files already downloaded and verified
	Total Len.:  50000 
 --------------------------------------------------

Loading CIFAR10  test  Dataset...
Files already downloaded and verified
	Total Len.:  10000 
 --------------------------------------------------
Num workers 11


In [4]:


####################################################################
# Neural Network Class
####################################################################

# Define the CNN model
# Define ConvBlocks to create a CNN

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_prob=0.3):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout2d(p=dropout_prob)  # Dropout para activaciones convolucionales
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout(x)  # Regularización adicional
        return x


# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        
        self.conv_layers = nn.Sequential(
                ConvBlock(3, 32, dropout_prob=0.05),
                ConvBlock(32, 64, dropout_prob=0.1),
                ConvBlock(64, 128, dropout_prob=0.2),
                ConvBlock(128, 256, dropout_prob=0.3),
                ConvBlock(256, 512, dropout_prob=0.4)
            )
        
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.relu = nn.ReLU()

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x


In [5]:
# Instantiating the network and printing its architecture
num_classes = 10
net = SimpleCNN(num_classes)
print(net)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Params: ", count_parameters(net))

####################################################################
# Training settings
####################################################################

# Training hyperparameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=10, min_lr=0.00001)
epochs = 100

SimpleCNN(
  (conv_layers): Sequential(
    (0): ConvBlock(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (dropout): Dropout2d(p=0.05, inplace=False)
    )
    (1): ConvBlock(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (dropout): Dropout2d(p=0.1, inplace=False)
    )
    (2): ConvBlock(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (pool): MaxPool2d(kernel_

In [6]:
trainer = CIFAR10_trainer(net, train_dataloader, test_dataloader, optimizer,criterion, epochs, lr_scheduler)

In [7]:
trainer.train()


---- Start Training ----


Test 0: 100%|██████████| 67/67 [00:01<00:00, 44.51batch/s]


[Epoch 1] Train Loss: 1.145813 - Test Loss: 0.953274 - Train Error: 64.69% - Test Error: 52.50%


Test 1: 100%|██████████| 67/67 [00:01<00:00, 34.46batch/s]

[Epoch 2] Train Loss: 0.928216 - Test Loss: 0.794818 - Train Error: 51.06% - Test Error: 44.20%


...

[Epoch 73] Train Loss: 0.114860 - Test Loss: 0.469265 - Train Error: 5.95% - Test Error: 17.49%



Test 73: 100%|██████████| 67/67 [00:01<00:00, 40.54batch/s]

[Epoch 74] Train Loss: 0.115949 - Test Loss: 0.480031 - Train Error: 5.99% - Test Error: 17.58%



Test 74: 100%|██████████| 67/67 [00:01<00:00, 38.64batch/s]

[Epoch 75] Train Loss: 0.111995 - Test Loss: 0.465089 - Train Error: 5.77% - Test Error: 17.57%

Early Stopping at epoch  74

BEST TEST ERROR:  17.45  in epoch  58
